In [1]:
import os
if "models" not in os.listdir("."):
    os.chdir("../..")

In [2]:
%load_ext autoreload
%autoreload 2
import penzai
import jax_smi
jax_smi.initialise_tracking()
from penzai import pz
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

In [3]:
from micrlhf.llama import LlamaTransformer
llama = LlamaTransformer.from_pretrained("models/gemma-2b-it.gguf", from_type="gemma", load_eager=True, device_map="tpu:0")

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("alpindale/gemma-2b")
tokenizer.padding_side = "right"

In [5]:
from sprint.icl_sfc_utils import Circuitizer

In [6]:
from sprint.task_vector_utils import load_tasks, ICLRunner
tasks = load_tasks()

fatal: destination path 'data/itv' already exists and is not an empty directory.


In [7]:
task_names = ["en_es", "antonyms", "person_profession"]

In [9]:
import numpy as np

for task_name in tasks:

    task = tasks[task_name]

    pairs = list(task.items())

    batch_size = 8
    n_shot=12
    if task_name.startswith("algo"):
        n_shot = 8
    max_seq_len = 128
    seed = 10

    prompt = "Follow the pattern:\n{}"

    runner = ICLRunner(task_name, pairs, batch_size=batch_size, n_shot=n_shot, max_seq_len=max_seq_len, seed=seed, prompt=prompt, use_same_examples=False, use_same_target=False)


    train_tokens = runner.get_tokens(
        runner.train_pairs, tokenizer
    )["input_ids"]

    tokens_wrapped = pz.nx.wrap(train_tokens, "batch", "seq")
    llama_inputs = llama.inputs.from_basic_segments(tokens_wrapped)

    logits = llama(llama_inputs)

    logits = logits.unwrap(
        "batch", "seq", "vocabulary"
    )

    decoded = [tokenizer.decode([x]) for x in logits[train_tokens == 3978].argmax(axis=-1)]

    sep_mask = train_tokens == 3978

    sep_mask = np.roll(sep_mask, 1, axis=1)

    tgt_tokens = train_tokens[sep_mask]

    hits = 0

    for p, t in zip(decoded, tgt_tokens):
        d = tokenizer.decode([t])

        if p.strip()[:3] == d.strip()[:3]:
            hits += 1

        
    print(task_name, hits / len(decoded))

location_continent 0.5729166666666666
football_player_position 0.22916666666666666
location_religion 0.71875
location_language 0.53125
person_profession 0.2604166666666667
location_country 0.22916666666666666
country_capital 0.6875
person_language 0.625
singular_plural 0.8645833333333334
present_simple_past_simple 0.8854166666666666
antonyms 0.71875
plural_singular 0.8958333333333334
present_simple_past_perfect 0.7708333333333334
present_simple_gerund 0.9375
en_it 0.5208333333333334
it_en 0.4583333333333333
en_fr 0.5104166666666666
en_es 0.6145833333333334
fr_en 0.5
es_en 0.5104166666666666
algo_last 0.84375
algo_first 0.953125
algo_second 0.40625


In [10]:
import numpy as np

for task_name in tasks:

    task = tasks[task_name]

    pairs = list(task.items())

    batch_size = 8
    n_shot=24
    if task_name.startswith("algo"):
        n_shot = 8
    max_seq_len = 256
    seed = 10

    prompt = "Follow the pattern:\n{}"

    runner = ICLRunner(task_name, pairs, batch_size=batch_size, n_shot=n_shot, max_seq_len=max_seq_len, seed=seed, prompt=prompt, use_same_examples=False, use_same_target=False)


    train_tokens = runner.get_tokens(
        runner.train_pairs, tokenizer
    )["input_ids"]

    tokens_wrapped = pz.nx.wrap(train_tokens, "batch", "seq")
    llama_inputs = llama.inputs.from_basic_segments(tokens_wrapped)

    logits = llama(llama_inputs)

    logits = logits.unwrap(
        "batch", "seq", "vocabulary"
    )

    decoded = [tokenizer.decode([x]) for x in logits[train_tokens == 3978].argmax(axis=-1)]

    sep_mask = train_tokens == 3978

    sep_mask = np.roll(sep_mask, 1, axis=1)

    tgt_tokens = train_tokens[sep_mask]

    hits = 0

    for p, t in zip(decoded, tgt_tokens):
        d = tokenizer.decode([t])

        if p.strip()[:3] == d.strip()[:3]:
            hits += 1

        
    print(task_name, hits / len(decoded))

location_continent 0.6458333333333334
football_player_position 0.328125
location_religion 0.7291666666666666
location_language 0.6822916666666666
person_profession 0.3385416666666667
location_country 0.3020833333333333
country_capital 0.65625
person_language 0.6197916666666666
singular_plural 0.8802083333333334
present_simple_past_simple 0.9479166666666666
antonyms 0.7239583333333334
plural_singular 0.8645833333333334
present_simple_past_perfect 0.7760416666666666
present_simple_gerund 0.9270833333333334
en_it 0.6354166666666666
it_en 0.5364583333333334
en_fr 0.5677083333333334
en_es 0.71875
fr_en 0.5885416666666666
es_en 0.5833333333333334
algo_last 0.84375
algo_first 0.953125
algo_second 0.40625
